## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-04-23-47-00 +0000,wsj,UPS Cargo Plane Crashes in Kentucky,https://www.wsj.com/business/logistics/ups-car...
1,2025-11-04-23-43-00 +0000,wsj,Private-Credit Earnings Ease Investor Concern ...,https://www.wsj.com/finance/private-credit-ear...
2,2025-11-04-23-41-00 +0000,wsj,Gold Edges Higher on Possible Technical Recovery,https://www.wsj.com/finance/commodities-future...
3,2025-11-04-23-37-07 +0000,cbc,Belgian airports closed after drone sightings,https://www.cbc.ca/news/world/belgium-brussels...
4,2025-11-04-23-32-53 +0000,nyt,Polls Have Shown Rising Support for California...,https://www.nytimes.com/2025/11/04/business/ca...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2375/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
82,trump,50
72,new,24
269,cheney,18
80,election,18
577,budget,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
317,2025-11-04-10-00-00 +0000,wsj,When the Supreme Court hears arguments over Pr...,https://www.wsj.com/politics/policy/whats-at-s...,135
141,2025-11-04-19-43-00 +0000,wsj,The Trump administration will comply with a co...,https://www.wsj.com/politics/policy/trump-thre...,116
319,2025-11-04-09-30-53 +0000,bbc,Trump backs Cuomo for New York City mayor and ...,https://www.bbc.com/news/articles/c231e284345o...,112
200,2025-11-04-17-31-00 +0000,wsj,Some Tuesday elections will test whether Trump...,https://www.wsj.com/politics/elections/divided...,110
149,2025-11-04-19-21-43 +0000,nypost,"Trump White House snubs ex-VP Dick Cheney, off...",https://nypost.com/2025/11/04/us-news/trump-wh...,108


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
317,135,2025-11-04-10-00-00 +0000,wsj,When the Supreme Court hears arguments over Pr...,https://www.wsj.com/politics/policy/whats-at-s...
327,85,2025-11-04-09-00-00 +0000,cbc,Carney's 1st budget calls for billions in new ...,https://www.cbc.ca/news/politics/carney-first-...
188,71,2025-11-04-17-56-00 +0000,wsj,The government pays out $100 billion a year in...,https://www.wsj.com/politics/policy/what-to-kn...
278,65,2025-11-04-11-52-00 +0000,wsj,"Dick Cheney, Powerful Former Vice President Wh...",https://www.wsj.com/politics/dick-cheney-power...
59,60,2025-11-04-22-27-44 +0000,nypost,NJ minor arrested in connection to bomb threat...,https://nypost.com/2025/11/04/us-news/new-jers...
110,52,2025-11-04-21-11-06 +0000,nypost,Andrew Cuomo calls race against Zohran Mamdani...,https://nypost.com/2025/11/04/us-news/cuomo-fr...
235,48,2025-11-04-15-01-47 +0000,nypost,France threatens to block Shein over sale of c...,https://nypost.com/2025/11/04/world-news/franc...
200,46,2025-11-04-17-31-00 +0000,wsj,Some Tuesday elections will test whether Trump...,https://www.wsj.com/politics/elections/divided...
52,43,2025-11-04-22-40-00 +0000,wsj,Major Tesla Investor Rejects Elon Musk’s $1 Tr...,https://www.wsj.com/business/autos/major-tesla...
297,41,2025-11-04-11-00-00 +0000,latimes,California voters head to the polls to decide ...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
